In [1]:
import torch
import os 
import sklearn 
import pandas as pd 
import numpy as np
from carla.data.catalog import CsvCatalog

/home/jacqueline/.local/share/virtualenvs/CARLA-7s4zdYsf/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/jacqueline/.local/share/virtualenvs/CARLA-7s4zdYsf/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/jacqueline/.local/share/virtualenvs/CARLA-7s4zdYsf/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint

[INFO] Using Python-MIP package version 1.12.0 [model.py <module>]


In [2]:
#TODO How to get access to test set from CARLA ? 
name='credit'
dataset = pd.read_csv(f'../data/{name}/{name}.csv')
            #TODO Better way for defining continous varaibles ?
continuous_wachter = dataset.drop(columns=['label']).columns
dataset = CsvCatalog(file_path=f'../data/{name}/{name}.csv',
                     continuous=continuous_wachter,
                     categorical=[],
                     immutables=[],
                     target='label',
                     scaling_method='Identity')
dataset._df_test

,label,x1,x2,x3,x4,x6,x5,x7
406,0.0,0.0,6.372431,-0.187021,1.360153,-0.473795,-3.342815,-5.988362
1093,0.0,1.0,5.248368,0.261969,2.197633,1.850286,4.508603,0.519989
9250,1.0,0.0,5.176884,-0.227096,1.569174,-2.058220,1.622966,-5.572757
5261,1.0,1.0,-6.907368,0.030302,-0.500954,1.363844,5.109764,5.306446
2563,1.0,0.0,-12.265844,-0.129698,0.820999,-1.554638,-2.621681,-6.115380
...,...,...,...,...,...,...,...,...
6415,1.0,0.0,-18.119290,-0.348884,-3.994172,-1.801456,-6.564125,-12.743086
1605,1.0,1.0,11.694004,0.068139,3.979982,3.876781,5.039257,3.695176
6135,1.0,0.0,-4.140342,-0.111427,-3.078975,-0.302476,-8.596558,2.786724
2180,0.0,0.0,3.621749,-0.274919,-3.251625,-0.136051,-4.266198,-10.372616


In [3]:
from string import digits
import re
model_name=['0']
accuracy=['0']
precision=['0']
recall=['0']
auc=['0']
df=pd.DataFrame([])

for name in os.listdir('./Model/'):
    #print(name.split('.')[0][-1].isdigit())
    #print(name)
    if name.split('.')[0][-1].isdigit() or name.startswith('Linear')or name.startswith('Forest'):
        full_name=name
        model=torch.load(f'./Model/{name}')
        name=name.split('_')[1]
        name=name.replace('.pth','')
        #print('FIRST',name)
        #print(name[:-1])
        if name[-1].isdigit():
            name=name[:-1]
        #print('Second',name)

        dataset = pd.read_csv(f'../data/{name}/{name}.csv')
        continuous_wachter = dataset.drop(columns=['label']).columns
        scaler='Identity'
        if name == 'economic':
            scaler='MinMax'

        dataset = CsvCatalog(file_path=f'../data/{name}/{name}.csv',
                     continuous=continuous_wachter,
                     categorical=[],
                     immutables=[],
                     target='label',
                     scaling_method=scaler)
        dataset=dataset._df_test
        data_x, data_y=dataset.drop(columns=['label']), dataset['label']
        data_pred= model.predict(data_x)
        model_name.append(full_name)
        print(data_pred.shape)
        print(data_y.shape)
        if len(data_pred.shape)>1:
            data_pred= np.argmax(data_pred, axis=1)

        accuracy.append(sklearn.metrics.accuracy_score(data_y,data_pred))
        precision.append(sklearn.metrics.precision_score(data_y,data_pred))
        recall.append(sklearn.metrics.recall_score(data_y,data_pred))
        auc.append(sklearn.metrics.roc_auc_score(data_y,data_pred))
df['model_name']=model_name
df['accuracy']=accuracy
df['precision']=precision
df['recall']=recall
df['auc'] =auc
df

(2500,)
(2500,)
(2500,)
(2500,)
(2500,)
(2500,)
(2500,)
(2500,)
(2500,)
(2500,)
(2500, 2)
(2500,)


/home/jacqueline/.local/share/virtualenvs/CARLA-7s4zdYsf/lib/python3.7/site-packages/carla/models/catalog/Linear_TORCH/model_linear.py:34: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  output = self.softmax(output)
/home/jacqueline/.local/share/virtualenvs/CARLA-7s4zdYsf/lib/python3.7/site-packages/carla/models/catalog/Linear_TORCH/model_linear.py:34: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  output = self.softmax(output)
/home/jacqueline/.local/share/virtualenvs/CARLA-7s4zdYsf/lib/python3.7/site-packages/carla/models/catalog/Linear_TORCH/model_linear.py:34: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  output = self.softmax(output)
/home/jacqueline/.local/share/virtualenvs/CARLA-7s4zdYsf/lib/python3.7/site-packages/carla/models/catalog/Linear_TORCH/model

(2500, 2)
(2500,)
(2500, 2)
(2500,)
(2500, 2)
(2500,)
(2500, 2)
(2500,)
(2500, 2)
(2500,)
(2500, 2)
(2500,)
(2500, 2)
(2500,)


/home/jacqueline/.local/share/virtualenvs/CARLA-7s4zdYsf/lib/python3.7/site-packages/carla/models/catalog/ANN_TORCH/model_ann.py:56: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  x = self.softmax(x)
/home/jacqueline/.local/share/virtualenvs/CARLA-7s4zdYsf/lib/python3.7/site-packages/carla/models/catalog/ANN_TORCH/model_ann.py:56: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  x = self.softmax(x)
/home/jacqueline/.local/share/virtualenvs/CARLA-7s4zdYsf/lib/python3.7/site-packages/carla/models/catalog/ANN_TORCH/model_ann.py:56: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  x = self.softmax(x)
/home/jacqueline/.local/share/virtualenvs/CARLA-7s4zdYsf/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision is ill-defin

(2500, 2)
(2500,)
(2500, 2)
(2500,)
(2500, 2)
(2500,)
(2500, 2)
(2500,)
(2500, 2)
(2500,)
(2500, 2)
(2500,)


/home/jacqueline/.local/share/virtualenvs/CARLA-7s4zdYsf/lib/python3.7/site-packages/carla/models/catalog/ANN_TORCH/model_ann.py:56: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  x = self.softmax(x)
/home/jacqueline/.local/share/virtualenvs/CARLA-7s4zdYsf/lib/python3.7/site-packages/carla/models/catalog/ANN_TORCH/model_ann.py:56: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  x = self.softmax(x)
/home/jacqueline/.local/share/virtualenvs/CARLA-7s4zdYsf/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/jacqueline/.local/share/virtualenvs/CARLA-7s4zdYsf/lib/python3.7/site-packages/carla/models/catalog/ANN_TO

(2500, 2)
(2500,)
(2500, 2)
(2500,)
(2500, 2)
(2500,)
(2500, 2)
(2500,)
(2500, 2)
(2500,)
(2500, 2)
(2500,)


/home/jacqueline/.local/share/virtualenvs/CARLA-7s4zdYsf/lib/python3.7/site-packages/carla/models/catalog/ANN_TORCH/model_ann.py:56: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  x = self.softmax(x)
/home/jacqueline/.local/share/virtualenvs/CARLA-7s4zdYsf/lib/python3.7/site-packages/carla/models/catalog/ANN_TORCH/model_ann.py:56: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  x = self.softmax(x)


,model_name,accuracy,precision,recall,auc
0,0,0,0,0,0
1,Forest_credit.pth,0.7976,0.813421,0.781447,0.797889
2,Forest_economic.pth,0.8164,0.765432,0.818096,0.816623
3,Forest_sanity-3-lin.pth,0.806,0.823626,0.788069,0.806351
4,Forest_sanity-3-non-add.pth,0.8608,0.926829,0.515837,0.75057
5,Forest_sanity-3-non-lin.pth,0.9708,0.978761,0.957576,0.969866
6,Linear_credit.pth,0.7488,0.775327,0.728879,0.749665
7,Linear_economic.pth,0.3624,0.337282,0.504664,0.380133
8,Linear_sanity-3-lin.pth,0.996,0.997671,0.994582,0.996049
9,Linear_sanity-3-non-add.pth,1,1,1,1


In [5]:
# Table to Latex 
# Get header
st=''
col=[]
name=[]
df=df.iloc[1:]
n_saver=[]
key=[]
for index, row in df.iterrows():
    #print(row['model_name'].split('_'))
    #if st is '':
    #    st += row['model_name'].split('_')[1].replace('.pth','')[:-1]
    #else:
    #    st += ' & '+row['model_name'].split('_')[1].replace('.pth','')[:-1]
    if row['model_name'].split('_')[1].replace('.pth','')[-1].isdigit():
        print('is Digit')
        data=row['model_name'].split('_')[0]+ row['model_name'].split('_')[1].replace('.pth','')[-1]
    else: 
        data=row['model_name'].split('_')[0]
    col.append(data)
    name.append(row['model_name'].split('_')[1].replace('.pth','')[:-1])
    key.append(row['model_name'])
st+= ' \\\\ '
col= np.unique(col)
name = np.unique(name[name!='credi'])
name=['economic','credit','sanity-3-lin','sanity-3-non-lin','sanity-3-non-add']
for n in name :
    st += ' & '+ n

print('col',col)
print('name',name)
print('key', key)
for i in range(0,len(col)):
    st += col[i] 
    num=''
    if col[i][-1].isdigit():
        num = col[i][-1]
    for j in range(0,len(name)):
        #print(''col[i])
        #print(name[i])
        for k in key: 
            print(f'{name[j]}{num}')
            if col[i] in k and f'{name[j]}{num}' in k: 
                t=k

        print(t)
        #print(f'{col[i]}_{name[i]}')
        #print(str(df[df['model_name'] ==f'{col[i]}_{name[j]}']['accuracy'].values[0]))
        #print(df[df['model_name'] ==f'{col[i]}_{name[i]}']['accuracy'])
        st += ' & ' + str(df[df['model_name'] ==f'{t}']['accuracy'].values[0])
    st+=' \\\\ '
print(st)

is Digit
is Digit
is Digit
is Digit
is Digit
is Digit
is Digit
is Digit
is Digit
is Digit
is Digit
is Digit
is Digit
is Digit
is Digit
col ['Forest' 'Linear' 'MLP0' 'MLP1' 'MLP2']
name ['economic', 'credit', 'sanity-3-lin', 'sanity-3-non-lin', 'sanity-3-non-add']
key ['Forest_economic.pth', 'Forest_sanity-3-lin.pth', 'Forest_sanity-3-non-add.pth', 'Forest_sanity-3-non-lin.pth', 'Linear_credit.pth', 'Linear_economic.pth', 'Linear_sanity-3-lin.pth', 'Linear_sanity-3-non-add.pth', 'Linear_sanity-3-non-lin.pth', 'MLP_credit0.pth', 'MLP_credit1.pth', 'MLP_credit2.pth', 'MLP_economic0.pth', 'MLP_economic1.pth', 'MLP_economic2.pth', 'MLP_sanity-3-lin0.pth', 'MLP_sanity-3-lin1.pth', 'MLP_sanity-3-lin2.pth', 'MLP_sanity-3-non-add0.pth', 'MLP_sanity-3-non-add1.pth', 'MLP_sanity-3-non-add2.pth', 'MLP_sanity-3-non-lin0.pth', 'MLP_sanity-3-non-lin1.pth', 'MLP_sanity-3-non-lin2.pth']
economic
economic
economic
economic
economic
economic
economic
economic
economic
economic
economic
economic
economic
